In [1]:
import asyncio

# for LLM
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

# AGI Agents
import AGI_agents as aa

# Apply the necessary setup for Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()

In [2]:
class CognitiveFeedbackRouter:
    def __init__(self, model_path: str = "llama-2-7b-chat.Q4_K_M.gguf", agents_dict: dict = {}):
        self.agents = agents_dict
        self.number = 0
        self.stored_input = ''
        self.urgent_task = asyncio.Event()
        self.sleeping = asyncio.Event()
        self.lock = asyncio.Lock()
        self.llm = None
        self.model_path = model_path

    async def wakeup(self):
        async with self.lock:  # Acquire the lock
            # Logic for wakeup
            print("Waking up...")
            self.llm = LlamaCpp(
                model_path="llama-2-7b-chat.Q4_K_M.gguf",
                max_tokens=100000)
            
            if self.urgent_task.is_set():
                print(f"I urgently need to take care of something.")
            self.sleeping.clear()
            print("Now I'm fully awaken.")

    async def deepsleep(self):
        async with self.lock:  # Acquire the lock
            # Logic for deepsleep
            print("Zzzzzz...")
            self.sleeping.set()
            await asyncio.sleep(5)  # Simulate some work
            asyncio.create_task(self.wakeup())

    async def get_user_input(self):
        while True:
            user_input = await asyncio.get_event_loop().run_in_executor(None, input, "Enter something: ")
            if self.sleeping.is_set():
                self.stored_input = user_input
            else:
                await self.agents['Human interaction']['class'].run(self.llm, user_input)

    async def decide_what_to_do(self):
        while True:
            if not self.sleeping.is_set():
                self.number += 1
                if self.stored_input:
                    await self.agents['Human interaction']['class'].run(self.llm, self.stored_input)
                    self.stored_input = None  # Clear stored input after processing
                    #elif self.number%5 == 0:
                    #    print(self.number)
                    #    await self.deepsleep()
                else:
                    print(self.number)
                    await self.agents['Contemplation']['class'].run()
            else:
                await asyncio.sleep(1)
            
            await self.decide_what_to_do()
    
    async def run(self):
        await self.wakeup()
        
        input_task = asyncio.create_task(self.get_user_input())
        
        await self.decide_what_to_do()


class HumanInteraction:
    def __init__(self):
        self.description = "Human interaction agent placeholder"

    async def run(self, llm, input):
        print(f"This is input: {input}.")
        prompt_template = PromptTemplate.from_template("Could you please answer this: {input}?")
        chain = prompt_template | llm
        result = chain.invoke({'input': input})
        print(f"This is the answer: {result}.")

class Contemplation:
    def __init__(self):
        self.description = "Self-contemplation placeholer"

    async def run(self):
        #print("Contemplating...")
        await asyncio.sleep(10)  # Simulate contemplation

agent_hint = HumanInteraction()
agent_cont = Contemplation()

global_agents = {
    'Human interaction': {'Description': 'Desc1', 'class': agent_hint}, 
    'Contemplation': {'Description': 'Desc2', 'class': agent_cont}
}

# Create an instance of the class and run it
router = CognitiveFeedbackRouter(agents_dict=global_agents)
asyncio.run(router.run())

Waking up...


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


Now I'm fully awaken.
1
2


Enter something:  What would be a good question?


This is input: What would be a good question?.
This is the answer: 
I'm happy to help! A good question for your critical thinking exercise could depend on the context and goals of the activity. Here are some examples of open-ended questions that could lead to interesting discussions and analysis:
1. What do you think is the most pressing social issue facing society today, and why? How would you address it?
2. Should there be limits to free speech, and if so, what might those limits be? Why or why not?
3. In what ways can technology improve our lives, and in what ways can it harm us? What are some potential solutions to mitigate the negative effects of technology?
4. How do you think humanity will change in the next 50 years? What advancements do you predict, and what challenges might we face?
5. What are some potential solutions to address climate change, and how can individuals contribute to reducing their carbon footprint?
6. Should there be a universal basic income for all citizens,

KeyboardInterrupt: 

In [3]:
llm = LlamaCpp(model_path="llama-2-7b-chat.Q4_K_M.gguf", max_tokens=100000)

class HumanInteraction:
    def __init__(self):
        self.description = "Human interaction agent placeholder"

    async def run(self, llm, input):
        print(f"This is input: {input}.")
        prompt_template = PromptTemplate.from_template("Could you please answer this: {input}?")
        chain = prompt_template | llm
        result = chain.invoke({'input': input})
        print(f"This is the answer {result}.")

class Contemplation:
    def __init__(self):
        self.description = "Self-contemplation placeholer"

    async def run(self):
        #print("Contemplating...")
        await asyncio.sleep(10)  # Simulate contemplation
agent_hint = HumanInteraction()
agent_cont = Contemplation()

global_agents = {
    'Human interaction': {'Description': 'Desc1', 'class': agent_hint}, 
    'Contemplation': {'Description': 'Desc2', 'class': agent_cont}
}

result = await global_agents['Human interaction']['class'].run(input='2+2', llm=llm)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


This is input: 2+2.
This is the answer .


In [8]:
prompt_template = PromptTemplate.from_template("Could you please answer this: {input}?")
chain = prompt_template | llm

In [16]:
input = "What do you find interesting?"

In [17]:
result = chain.ainvoke({'input': input})

Llama.generate: prefix-match hit
C:\Users\siwiakma\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\langchain\llms\llamacpp.py:352: RuntimeWarning: coroutine 'AsyncCallbackManagerForLLMRun.on_llm_new_token' was never awaited
  run_manager.on_llm_new_token(
Llama.generate: prefix-match hit


In [20]:
chain

PromptTemplate(input_variables=['input'], template='Could you please answer this: {input}?')
| LlamaCpp(client=<llama_cpp.llama.Llama object at 0x00000288F7CEF6D0>, model_path='llama-2-7b-chat.Q4_K_M.gguf')

In [19]:
output

LLMResult(generations=[[Generation(text="\n\nI'm just an AI, I don't have personal preferences or interests like humans do. However, I can provide information on a wide range of topics including science, technology, history, culture, and more. Is there something specific you would like to know or discuss?")]], llm_output=None, run=[RunInfo(run_id=UUID('808c103a-019b-4130-8d65-edff1cc8bb54'))])

"\nI'm getting this error message while trying to execute a cell in Jupyter notebook. The cell contains the following line:\n`raw_input(y)`\nI'm using Python 3.7.4 and Jupyter Notebook 2.1.0. I believe that the issue might be related to the fact that `raw_input()` is deprecated in Python 3.7, but I couldn't find any solution so far. Any help would be greatly appreciated!\n\n\n\n"

In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
import re
from AGI_templates import interaction_template

# Define the custom prompt template
class CustomPromptTemplate(StringPromptTemplate):
    template = interaction_template  # Your provided template

    def format(self, **kwargs) -> str:
        # Include logic for formatting the prompt with tools, input, etc.
        # ...

# Define tools
search = SerpAPIWrapper()  # Assuming a search tool
tools = [Tool(name="Search", func=search.run, description="...")]

# Define the output parser
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str):
        # Logic to parse the output and decide if all angles are covered
        # ...

# Set up LLM and the agent
prompt = CustomPromptTemplate(tools=tools, ...)
llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = LLMSingleActionAgent(llm_chain=llm_chain, output_parser=CustomOutputParser(), ...)

# Run the agent
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
result = agent_executor.run("Your input here")

In [ ]:
class HumanInteractionAgent:
    def __init__(self, llm, prompt_template, output_parser, tools):
        self.llm = llm
        self.prompt_template = prompt_template
        self.output_parser = output_parser
        self.tools = tools
        self.context = []  # For storing intermediate steps

    def receive_input(self, user_input):
        # Process the initial user input
        ...

    def generate_prompt(self):
        # Use the prompt template to format the input and context
        ...

    def consult_llm(self, formatted_prompt):
        # Send the formatted prompt to the LLM and get a response
        ...

    def parse_output(self, llm_output):
        # Interpret the LLM's response using the output parser
        ...

    def decide_action(self, parsed_output):
        # Decide what action to take based on the parsed output
        ...

    def perform_action(self, action, action_input):
        # Perform the suggested action using one of the tools
        ...

    def iterate_interaction(self):
        # Repeat the process (prompt generation, LLM consultation, etc.)
        ...

    def conclude_interaction(self):
        # Finalize and format the response, and conclude the interaction
        ...

# Usage
agent = CustomLangChainAgent(llm, prompt_template, output_parser, tools)
response = agent.receive_input("User's question")

In [3]:
import os
print(os.listdir(os.getcwd()))

['.chainlit', '.ipynb_checkpoints', '.jupyter', 'AGI.pdf', 'AGI_templates.py', 'CFA.ipynb', 'CFR.ipynb', 'chainlit.md', 'CodeLlama', 'codellama-7b.Q4_K_M.gguf', 'cypher.py', 'graph_document.py', 'Graph_Layer.ipynb', 'graph_store.py', 'langchain_neo4j.ipynb', 'Library', 'llama-2-7b-chat.Q4_K_M.gguf', 'llama-2-7b-chat.Q6_K.gguf', 'LLM.ipynb', 'neo4j', 'neo4j_graph.py', 'SRI.ipynb', '__pycache__']
